# Advanced Generative Q&A Chatbot Using distilBERT model - UX Frontend using Gradio and hosted on Hugging Face Spaces
## AAI-520 Team 3 Final Project

Team 3 Members:  Tyler Foreman, Ahmed Ahmed, Tursun Alkam

Date:  October 23, 2023

GitHub Repository: https://github.com/t4ai/aai520-team3-final

Live hosted version: https://huggingface.co/spaces/t4ai/T3Soft-QA-Chatbot

In [36]:
import re
import os
from dotenv import load_dotenv
import json
import gradio as gr
import random 
import time 
import requests

from transformers import BertModel, BertTokenizerFast, AdamW
import tensorflow as tf

In [84]:
# load API key
load_dotenv(override=True)
if not os.getenv("HF_API_KEY"):
    raise ValueError("HF_API_KEY must be set")
hf_key = os.getenv('HF_API_KEY')

In [78]:
# setup model inference API with authorization token
API_URL = "https://api-inference.huggingface.co/models/t4ai/distilbert-finetuned-t3-qa"
headers = {"Authorization": "Bearer " + hf_key }

# helper function to call inference API and return response
def query_model(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

In [79]:
query_model({"inputs": {"question": "what is my name?", "context": "hello my name is John and I live in San Francisco"}})

{'score': 0.9349796772003174, 'start': 17, 'end': 21, 'answer': 'John'}

## Build UI Using Gradio for hosting on HuggingFace Spaces

Leverage gradio components to construct a chatbot interface along with location to input context/document text.  

In [96]:
# contruct UI using Gradio
_booted = False
with gr.Blocks() as demo:
    
    with gr.Row():
        with gr.Column(scale=1):
            context = gr.Textbox(label="Document Text", lines=25)
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(label="T3Soft Bot", value=[(None, "Welcome! I am your QA assistant."), (None, "Please paste your document content in the panel to the left."), (None, "Then submit questions below!")])
            with gr.Row():
                with gr.Column(scale=6):
                    msg = gr.Textbox(label="Ask your question")            
                with gr.Column(scale=1):
                    gr.Button("Send")
            clear = gr.ClearButton([msg, chatbot])
    
    def respond(message, context, chat_history):
        
        if(len(context) == 0):
            bot_message = "Hm, I don't see any document text, please paste in the box on the left."
        else:    
            query_bot = query_model({"inputs": {"question": message, "context": context}})
            if(len(query_bot) and ("answer" in query_bot)) and (query_bot['score'] > 0.1):
                bot_message = query_bot['answer'] + '(' + str(round(query_bot['score'], 2)) + ')'
            else:
                #bot_message = "I'm having trouble with this question, please try rewording."
                bot_message = random.choice(["I'm having trouble with this question, please try rewording and make sure it is relevant to the document.", "Hm, I'm having trouble finding the answer to that.  Can you reword the question?", "Sorry, I can't find the answer to this question."])
        
        
        chat_history.append((message, bot_message))
        time.sleep(2)
        return "", context, chat_history

    msg.submit(respond, [msg, context, chatbot], [msg, context, chatbot])

demo.launch()



Running on local URL:  http://127.0.0.1:7898

To create a public link, set `share=True` in `launch()`.
